In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv

np.random.seed(0)

In [ ]:
def get_radar(xpos_pred):
    """Return Predicted Horizontal Distance and Measured Distance by Radar."""
    xvel_w = np.random.normal(0, 5)   # xvel_w: system noise of horizontal velocity [m/s].
    xvel_true = 100 + xvel_w          # xvel_true: true horizontal velocity [m/s].

    ypos_w = np.random.normal(0, 10)  # ypos_w: system noise of vertical position [m].
    ypos_true = 1000 + ypos_w         # ypos_true: true vertical position [m].

    xpos_pred = xpos_pred + xvel_true * time_step              # xpos_pred: predicted horizontal distance [m].

    rpos_v = xpos_pred * np.random.normal(0, 0.05)             # rpos_v: measurment noise of distance from radar.
    rpos_meas = np.sqrt(xpos_pred**2 + ypos_true**2) + rpos_v  # r: measured distance [m] (observable).

    return rpos_meas, xpos_pred

In [ ]:
def Hjacob(x, H):
    H[0][0] = x[0] / np.sqrt(x[0]**2 + x[2]**2)
    H[0][1] = 0
    H[0][2] = x[2] / np.sqrt(x[0]**2 + x[2]**2)
    return H

In [ ]:
def kalman_filter(z_meas, x_esti, P):
    """Kalman Filter Algorithm."""
    # (1) Prediction.
    x_pred = A @ x_esti
    P_pred = A @ P @ A.T + Q

    # (2) Kalman Gain.
    K = P_pred @ H.T @ inv(H @ P_pred @ H.T + R)

    # (3) Eastimation.
    x_esti = x_pred + K @ (z_meas - H @ x_pred)

    # (4) Error Covariance.
    P = P_pred - K @ H @ P_pred

    return x_esti, P

In [ ]:
# Input parameters.
time_end = 20
time_step = 0.05

In [ ]:
time = np.arange(0, time_end, time_step)
n_samples = len(time)

In [ ]:
# Initialization for system model.
# Define matrix with rank 2 for matrix operation.
dt = time_step
A = np.eye(3) + dt * np.array([[0, 1, 0],
                               [0, 0, 0],
                               [0, 0, 0]])
H = np.zeros((1, 3))
Q = np.array([[0, 0, 0],
              [0, 0.001, 0],
              [0, 0, 0.001]])
R = np.array([[10]])

# Initialization for estimation.
x_0 = np.array([0, 90, 1100])  # [horizontal position, horizontal velocity, vertical position].
P_0 = 10 * np.eye(3)

In [ ]:
xpos_esti_save = np.zeros(n_samples)
ypos_esti_save = np.zeros(n_samples)
rpos_esti_save = np.zeros(n_samples)
xvel_esti_save = np.zeros(n_samples)
rpos_meas_save = np.zeros(n_samples)

In [ ]:
xpos_pred = 0
x_esti, P = None, None
for i in range(n_samples):
    z_meas, xpos_pred = get_radar(xpos_pred)
    if i == 0:
        x_esti, P = x_0, P_0
    else:
        H = Hjacob(x_esti, H)
        x_esti, P = kalman_filter(z_meas, x_esti, P)

    xpos_esti_save[i] = x_esti[0]
    ypos_esti_save[i] = x_esti[2]
    rpos_esti_save[i] = np.sqrt(x_esti[0]**2 + x_esti[2]**2)
    xvel_esti_save[i] = x_esti[1]
    rpos_meas_save[i] = z_meas

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(10, 30))

plt.subplot(4, 1, 1)
plt.plot(time, xpos_esti_save, 'bo-', label='Estimation (EKF)')
plt.legend(loc='upper left')
plt.title('Horizontal Distance: Esti. (EKF)')
plt.xlabel('Time [sec]')
plt.ylabel('Horizontal Distance [m]')

plt.subplot(4, 1, 2)
plt.plot(time, ypos_esti_save, 'bo-', label='Estimation (EKF)')
plt.legend(loc='upper left')
plt.title('Vertical Distance: Esti. (EKF)')
plt.xlabel('Time [sec]')
plt.ylabel('Vertical Distance [m]')

plt.subplot(4, 1, 3)
plt.plot(time, rpos_meas_save, 'r*--', label='Measurements', markersize=10)
plt.plot(time, rpos_esti_save, 'bo-', label='Estimation (EKF)')
plt.legend(loc='upper left')
plt.title('Radar Distance: Meas. v.s. Esti. (EKF)')
plt.xlabel('Time [sec]')
plt.ylabel('Radar Distance [m]')

plt.subplot(4, 1, 4)
plt.plot(time, xvel_esti_save, 'bo-', label='Estimation (EKF)')
plt.legend(loc='upper left')
plt.title('Horizontal Velocity: Esti. (EKF)')
plt.xlabel('Time [sec]')
plt.ylabel('Horizontal Velocity [m/s]')

plt.savefig('png/radar_ekf.png')